<a href="https://colab.research.google.com/github/cantalk-bolt/crypto-trading-bot/blob/main/bist_veri_cekme_ema_rsi_uyumsuzluk_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [3]:
!pip install git+https://github.com/rongardF/tvdatafeed
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
import requests
import warnings

warnings.simplefilter(action='ignore')

# Telegram bilgilerinizi buraya girin
TELEGRAM_TOKEN = '7062647054:AAFox0SMNLvZ1XihCI00_jZd_cvbRWEyzpk'  # Örnek: '123456789:ABCdefGHI_jklMNOpqrSTUvwxYZ'
TELEGRAM_CHAT_ID = '1265605289'  # Örnek: '-1001234567890'

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    data = {"chat_id": TELEGRAM_CHAT_ID, "text": message}
    requests.post(url, data=data)

def ema(series, length):
    return series.ewm(span=length, adjust=False).mean()

def rsi_func(series, length=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=length - 1, adjust=False).mean()
    ema_down = down.ewm(com=length - 1, adjust=False).mean()
    rs = ema_up / ema_down
    return 100 - (100 / (1 + rs))

def detect_bullish_divergence(df, rsi_col='RSI', left=5, right=5, range_lower=5, range_upper=60):
    df = df.copy()
    df['rsi_pivotlow'] = df[rsi_col][::-1].rolling(left + right + 1, center=True).apply(
        lambda x: x[right] == min(x), raw=True)[::-1].astype(bool)

    df['price_pivotlow'] = df['low'][::-1].rolling(left + right + 1, center=True).apply(
        lambda x: x[right] == min(x), raw=True)[::-1].astype(bool)

    bullish_div = []
    for i in range(len(df)):
        if df['rsi_pivotlow'].iloc[i]:
            for j in range(range_lower, range_upper + 1):
                if i - j >= 0 and df['rsi_pivotlow'].iloc[i - j]:
                    rsi_now = df[rsi_col].iloc[i]
                    rsi_prev = df[rsi_col].iloc[i - j]
                    price_now = df['low'].iloc[i]
                    price_prev = df['low'].iloc[i - j]

                    if rsi_now > rsi_prev and price_now < price_prev:
                        bullish_div.append(True)
                        break
            else:
                bullish_div.append(False)
        else:
            bullish_div.append(False)
    df['bullish_div'] = bullish_div + [False] * (len(df) - len(bullish_div))
    return df

def apply_strategy(data):
    df = data.copy()
    df['ema100'] = ema(df['close'], 100)
    df['ema200'] = ema(df['close'], 200)
    df['RSI'] = rsi_func(df['close'], 14)
    df = detect_bullish_divergence(df)

    df['LongEntry'] = (df['ema100'] > df['ema200']) & (df['bullish_div']) & (df['close'] < df['ema100'])
    return df

# Hisseler
Hisseler = ['AKBNK', 'THYAO', 'SISE', 'ASELS', 'KRDMD', 'IHLAS', 'TUKAS', 'HEKTS', 'VESBE', 'GWIND']  # Devamı eklenebilir

tv = TvDatafeed()
Titles = ['Hisse Adı', 'Son Fiyat', 'Long Sinyali']
df_signals = pd.DataFrame(columns=Titles)

for hisse in Hisseler:
    try:
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=Interval.in_1_hour, n_bars=250)
        data = data.reset_index()
        result = apply_strategy(data)
        last_row = result.iloc[-1]

        if last_row['LongEntry']:
            df_signals.loc[len(df_signals)] = [hisse, last_row['close'], True]
            print([hisse, last_row['close'], True])
    except Exception as e:
        print(f"{hisse} için hata: {e}")
        continue

# === Telegram Mesajı Gönder ===
long_sinyalli_hisseler = df_signals[df_signals['Long Sinyali'] == True]

if not long_sinyalli_hisseler.empty:
    mesaj = "📈 Saatlik Long Sinyali Gelen Hisseler:\n"
    for _, row in long_sinyalli_hisseler.iterrows():
        mesaj += f"• {row['Hisse Adı']} - Fiyat: {row['Son Fiyat']:.2f} ₺\n"
else:
    mesaj = "⛔ Bu saatlik periyotta long sinyali gelen hisse yok."

send_telegram_message(mesaj)


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-jjrjedsc
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-jjrjedsc
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


['IHLAS', np.float64(2.34), True]


In [ ]:
!pip install tradingview-ta
from tradingview_ta import TA_Handler, Interval, Exchange
# ... (rest of the code) ...

tv = TvDatafeed()
handler = TA_Handler(
    symbol="US500",
    screener="america",
    exchange="CME",
    interval=Interval.INTERVAL_1_HOUR
)

Hisseler = handler.get_analysis().summary
# Assuming 'Hisseler' is a list of symbols, proceed with your analysis

# ... (rest of your code) ...

Exception: Exchange or symbol not found.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. To run all the code in the notebook, select **Runtime** > **Run all**. To run the code cells one at a time, hover over each cell and select the **Run cell** icon.

![Run cell icon](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/images/beginner/run_cell_icon.png?raw=1)

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [ ]:
tf.nn.softmax(predictions).numpy()

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [ ]:
model.fit(x_train, y_train, epochs=5)

The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
